Image classification with CNNs
================

The goal of this exercise is to implement a specific CNN architecture with PyTorch and train it on the CIFAR-10 image classification dataset. We will start by introducing the dataset and then implement a `nn.Module` and a useful `Solver` class. Seperating the model from the actual training has proven itself as a sensible design decision. By the end of this exercise you should have succesfully trained your (possible) first CNN model and have a boilerplate `Solver` class which you can reuse for the next exercise and your future research projects.

For an inspiration on how to implement a model or the solver class you can have a look at [these](https://github.com/pytorch/examples) PyTorch examples.

In [1]:
import numpy as np
import os
from random import choice
from string import ascii_uppercase
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from yz.data_utils import get_Cancer_datasets
from yz.solver import Solver
from yz.data_utils import get_balanced_weights
from torchvision import models
import torch.nn as nn
import pandas as pd
from bayes_opt import BayesianOptimization

csv_full_name = '~/dl4cvproject/data/train.csv'
img_folder_full_name = '~/dl4cvproject/data/train256'
csv_full_name = os.path.expanduser(csv_full_name)
img_folder_full_name = os.path.expanduser(img_folder_full_name)

csv_full_name_test = '~/dl4cvproject/data/test.csv'
img_folder_full_name_test = '~/dl4cvproject/data/test256'
csv_full_name_test = os.path.expanduser(csv_full_name_test)
img_folder_full_name_test = os.path.expanduser(img_folder_full_name_test)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
train_data, val_data, test_data, train_label_list = get_Cancer_datasets(csv_full_name=csv_full_name,img_folder_full_name=img_folder_full_name)
test_X, csv_test = get_Cancer_datasets(csv_full_name=csv_full_name_test,img_folder_full_name=img_folder_full_name_test, mode='upload')
print("Train size: %i" % len(train_data))
print("Val size: %i" % len(val_data))
print("Test size: %i" % len(test_data))
print("upload size: {}", len(test_X))

100%|█████████▉| 18563/18577 [00:47<00:00, 390.82it/s]

transforming...



100%|██████████| 18577/18577 [00:00<00:00, 1670259.07it/s]

Done transforming...
Getting labels
submasking...
num_training:13000



  0%|          | 44/12386 [00:00<00:28, 436.44it/s]

OK...



100%|█████████▉| 18563/18577 [01:00<00:00, 309.09it/s][A
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 12386/12386 [00:31<00:00, 387.09it/s]


transforming...
Done transforming...
Train size: 13000
Val size: 1857
Test size: 3720
upload size: {} 12386


In [3]:
if torch.cuda.is_available():
    print('Cuda available')
else:
    print('Cuda not available :(---(')

Cuda not available :(---(


In [4]:
def target(factor, batch_size, lr_const, lr_exp, weight_decay_const, weight_decay_exp, num_epochs):

    batch_size = int(batch_size)
    num_epochs = int(num_epochs)
    lr_const = int(lr_const)
    weight_decay_const = int(weight_decay_const)
    
    #training
    weights = get_balanced_weights(label_list=train_label_list, num_classes=14, factor=factor)
    sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=sampler, num_workers=8)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=8)
    
    model = models.resnet18(pretrained=True)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, 14)

    lr = lr_const * np.power(10, lr_exp)
    weigth_decay = weight_decay_const * np.power(10, weight_decay_exp)    
    solver = Solver(optim_args={"lr":lr, "weight_decay":weigth_decay})
    solver.train(model, train_loader, val_loader, log_nth=1, num_epochs=num_epochs)
    
    #compute local prediction acc
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=False, num_workers=8)
    scores = []
    for inputs, target in tqdm(test_loader):
        inputs, targets = Variable(inputs), Variable(target)
        if torch.cuda.is_available:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        scores.extend((preds == targets).data.cpu().numpy())
        
    test_acc = np.mean(scores)
    
    ## generate submission file: submissions/res18_acc_randomsuffix.csv
    try:
        del csv_test['age']
    except KeyError as e:
        print(e)
    try:
        del csv_test['gender']
    except KeyError as e:
        print(e)
    try:
        del csv_test['view_position']
    except KeyError as e:
        print(e)
    try:
        del csv_test['image_name']
    except KeyError as e:
        print(e)
    try:
        del csv_test['detected']
    except KeyError as e:
        print(e)
        
    pred_set = set()
    for i in tqdm(range(len(test_X))):
        tmp_pred_list = [0] * 14
        inputs = test_X[i]
        inputs = Variable(inputs.unsqueeze(0))
        if torch.cuda.is_available:
            inputs = inputs.cuda()
        for trial in range(1):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            int_label = preds.data.cpu().numpy().tolist()[0]
            tmp_pred_list[int_label] += 1
        
        pred = tmp_pred_list.index(max(tmp_pred_list))
        str_pred = 'class_' + str(pred + 1)
        detected.append(str_pred)
    
    file_name = 'submissions/res18_' + '{:.5f}'.format(test_acc) + '_' + ''.join(choice(ascii_uppercase) for i in range(7)) + '.csv'
    csv_test['detected'] = pd.Series(detected)
    csv_test.to_csv(file_name, index=False)
    
    return test_acc

    

## Bayesian Optimization

In [5]:
bo = BayesianOptimization(target, {'factor':(0.5, 1), 'batch_size':(40, 100),
                                   'lr_const':(1, 10), 'lr_exp':(-3, -7),
                                   'weight_decay_const':(1, 10), 'weight_decay_exp':(-1, -6),
                                   'num_epochs':(1,2)})

In [6]:
bo.maximize(init_points=2, n_iter=0, acq='ucb', kappa=5)

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   batch_size |    factor |   lr_const |    lr_exp |   num_epochs |   weight_decay_const |   weight_decay_exp | 
weights: [0.058539683853059743, 0.046447823506344618, 0.014026381600760134, 0.032421512927313781, 0.022953674618201931, 0.031499957690940307, 0.022213382799792878, 0.062275084880815082, 0.16067922500953394, 0.089447635120552843, 0.041816266520847266, 0.027518030283374639, 0.045312804726346412, 0.051939374156584117]
equivalent_num:
17.7375242075
21.3659988129
55.9792889686
28.530931376
37.6669800485
29.2004607795
38.6734994544
16.8765480027
7.87328202547
12.612116552
23.2498441856
32.5538298252
21.7954590734
19.5292046829


  0%|          | 0/1 [00:00<?, ?it/s]

START TRAIN.
[Iteration 1 / 184] TRAIN loss: 2.955218
[Iteration 2 / 184] TRAIN loss: 3.091810
[Iteration 3 / 184] TRAIN loss: 3.658175
[Iteration 4 / 184] TRAIN loss: 3.483354
[Iteration 5 / 184] TRAIN loss: 3.091158
[Iteration 6 / 184] TRAIN loss: 2.994924
[Iteration 7 / 184] TRAIN loss: 2.903262
[Iteration 8 / 184] TRAIN loss: 2.845353
[Iteration 9 / 184] TRAIN loss: 2.687860
[Iteration 10 / 184] TRAIN loss: 2.663827
[Iteration 11 / 184] TRAIN loss: 2.738294
[Iteration 12 / 184] TRAIN loss: 2.464789
[Iteration 13 / 184] TRAIN loss: 2.865408


Process Process-8:
Process Process-6:
Process Process-5:
Process Process-4:
Process Process-3:
Process Process-1:
Process Process-7:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/hpc/pr92no/ga42cih2/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in

KeyboardInterrupt: 